### Notebook builds the User Table

In [2]:
import pandas as pd
from glob import glob as g
from pathlib import Path
import re

In [3]:
def fill_missing_annots(df):
    child_set = set(df['child'].tolist())
    temp_df = df[~df['parent'].isin(child_set)].copy()
    temp_df['child'] = temp_df['parent']
    temp_df = temp_df.drop_duplicates()
    return temp_df

In [11]:
# Set patterns
timestamp_pattern = "[[](\d\d:\d\d)[]]"
user_pattern = "[[]\d\d:\d\d[]]\s[<](.*?)[>]"
message_pattern = ".*[<].*[>](.*)"
date_pattern = "(\d\d\d\d-\d\d-\d\d)"
# Get files

# Train
filepath = '../data/'
subfolders = ["train", "test", "dev"]

raw_df_dict = {}
annot_dict = {}
user_set = set()
for subfolder in subfolders:
    filelist_ascii = g(filepath + subfolder + '/*ascii.txt')
    #filelist_annot = g(filepath + subfolder + '/*annotation.txt')
    for filename in filelist_ascii:
        filename_annot = filename.replace('ascii', 'annotation')
        data = pd.read_csv(filename, header = None, names = ['raw'], delimiter="\t")
        data['timestamp'] = data['raw'].str.extract(timestamp_pattern, expand=True)
        data.loc[data['timestamp'].isnull(), 'timestamp'] = "System"
        data['user'] = data['raw'].str.extract(user_pattern, expand=True)
        data.loc[data['user'].isnull(), 'user'] = "System"

        user_set.update(data['user'].values.tolist())

        data['message'] = data['raw'].str.extract(message_pattern, expand=True)
        data['file_ind'] = data.index.values
        data['file_ind'] = data['file_ind'].astype(int)
        #data['date'] = Path(filename).stem.split('_')[0]
        temp_date = re.search(date_pattern, filename).group(1)
        data['date'] = temp_date
        #data['uuid'] = data.apply(lambda row: generate_uuid(row), axis=1)

        raw_df_dict[temp_date] = data.copy()
        #merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']
        #merged_data = pd.merge(merged_data, merged_data[['file_ind', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['', '_parent'])
        #merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']

In [12]:
print(len(user_set))

17779


In [13]:
print(len(user_set))

17779


In [15]:
user_df = pd.DataFrame.from_dict({
    'user_txt': list(user_set),
    'user_id': list(range(0, len(user_set)))
})

In [17]:
user_df.shape

(17779, 2)

In [18]:
user_df.to_csv('users.csv')